In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import timeit
import psutil
import json

weight_external = 0.3
weight_performance = 0.7

In [9]:
#Importing Dataset
classDataset = "https://raw.githubusercontent.com/Akshay-De-Silva/ml_apis/main/heart.csv"
dataset = pd.read_csv(classDataset)

#relabel values in columns to be numeric
label_encoder = LabelEncoder()
dataset['Sex'] = label_encoder.fit_transform(dataset['Sex'])
dataset['ChestPainType'] = label_encoder.fit_transform(dataset['ChestPainType'])
dataset['RestingECG'] = label_encoder.fit_transform(dataset['RestingECG'])
dataset['ExerciseAngina'] = label_encoder.fit_transform(dataset['ExerciseAngina'])
dataset['ST_Slope'] = label_encoder.fit_transform(dataset['ST_Slope'])

X=dataset[['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol',
           'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope']].values
y=dataset[['HeartDisease']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
training_time = timeit.timeit(lambda: logistic_regression.fit(X_train, y_train), number=1)
testing_time = timeit.timeit(lambda: logistic_regression.predict(X_test), number=1)

ram_usage_bytes = psutil.virtual_memory().used
ram_usage = ram_usage_bytes / (1024 ** 2)

y_pred = logistic_regression.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1Score = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [11]:
max_time = 3
min_time = 0.005
max_gpu = 2000
min_gpu = 1000
max_cpu = 30
min_cpu = 15
max_ram = 2000
min_ram = 1000

max_acc = 0.97
min_acc = 0.75
max_f1 =0.95
min_f1 = 0.7

gpu_usage = 1100
cpu_usage = 18

tp_cost = 1/13
tn_cost = 1/13
fp_cost = 4/13
fn_cost = 7/13

conf_matrix = confusion_matrix(y_test, y_pred)
tp, fp, fn, tn = conf_matrix.ravel()
f1ci = (2*tp_cost*tp)/(2*tp_cost*tp + fp_cost*fp + fn_cost*fn)

bati = (weight_performance * ((accuracy - min_acc) / (max_acc - min_acc))) \
        + (weight_external * (1 - ((training_time - min_time) / (max_time - min_time))))

raf1_cpu = (weight_performance * ((f1Score-min_f1)/(max_f1-min_f1)) + (weight_external * (1-((cpu_usage-min_cpu)/(max_cpu-min_cpu)))))
raf1_gpu = (weight_performance * ((f1Score-min_f1)/(max_f1-min_f1)) + (weight_external * (1-((gpu_usage-min_gpu)/(max_gpu-min_gpu)))))
raf1_ram = (weight_performance * ((f1Score-min_f1)/(max_f1-min_f1)) + (weight_external * (1-((ram_usage-min_ram)/(max_ram-min_ram)))))

F1-Cost Index: 0.45
BATI = 0.61
Resource Adjusted F1 Score (CPU): 0.70
Resource Adjusted F1 Score (GPU): 0.73
Resource Adjusted F1 Score (RAM): 0.78
Accuracy: 0.85
F1 Score: 0.86
Precision: 0.91
Recall: 0.82


In [12]:
def class_to_json(f1_cost_index, bati, raf1_cpu, raf1_gpu, raf1_ram, accuracy, f1Score, precision, recall):
    output = {
        "BATI": round(bati,2),
        "RAF1_CPU" : round(raf1_cpu,2),
        "RAF1_GPU" : round(raf1_gpu,2),
        "RAF1_RAM" : round(raf1_ram,2),
        "F1CI" : round(f1ci,2),
        "BRMSETI" : 0,
        "RARS_CPU" : 0,
        "RARS_GPU" : 0,
        "RARS_RAM" : 0,
        "RMSPECI" : 0,
        "Accuracy" : round(accuracy,2),
        "F1Score" : round(f1Score,2),
        "Precision" : round(precision,2),
        "Recall" : round(recall,2),
        "MSE" : 0,
        "RSME" : 0,
        "MAE" : 0,
        "R2" : 0
    }
    return json.dumps(output, indent=4)

In [13]:
output = class_to_json(f1ci, bati, raf1_cpu, raf1_gpu, raf1_ram, accuracy, f1Score, precision, recall)
print(output)

{
    "BATI": 0.61,
    "RAF1_CPU": 0.7,
    "RAF1_GPU": 0.73,
    "RAF1_RAM": 0.78,
    "F1CI": 0.45,
    "BRMSETI": 0,
    "RARS_CPU": 0,
    "RARS_GPU": 0,
    "RARS_RAM": 0,
    "RMSPECI": 0,
    "Accuracy": 0.85,
    "F1Score": 0.86,
    "Precision": 0.91,
    "Recall": 0.82,
    "MSE": 0,
    "RSME": 0,
    "MAE": 0,
    "R2": 0
}
